# Data Engineering / Content Extraction

In [42]:
import pandas as pd
import re

df = pd.read_csv(r"data/user_ad_content.csv", encoding="utf-8")
df.head()

,url,location,username,img_urls,img_saved,title,Joined,Bumped,stars,self_descriptors,...,Hair Length,Grooming,Chest,Piercings,Tattoos,Smoke?,reviews,email,link,Created
0,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Karmen2020,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Karmen2020.00.jpg', 'Karmen2020.01.jpg', 'Ka...",New Girl Karmen,"July 14, 2020",1 hour(s) ago,0.0,"['Escort', 'Escort', 'Female', '25', 'Mixed', ...",...,NaN,NaN,NaN,check_box_outline_blank,check_box_outline_blank,check_box_outline_blank,[],NaN,NaN,NaN
1,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Minx_MsMariah,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Minx_MsMariah.00.jpg', 'Minx_MsMariah.01.jpg...",Secret Sunday Shenanigans,"January 27, 2019",1 hour(s) ago,5.0,"['Escort', 'Escort', 'Female', '45', 'White', ...",...,NaN,Bald,36 - 38 DD,check_box,check_box,check_box_outline_blank,"[{'by': 'Chowda', 'reviewer_reviews': '7', 'Jo...",msmariahmoon@protonmail.com,NaN,NaN
2,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Abigail_Reign,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Abigail_Reign.00.jpg', 'Abigail_Reign.01.jpg...",Save Us Time And Read My Ad In Its Entirety,"May 12, 2020",1 hour(s) ago,4.5,"['Escort', 'Escort', 'Female', '30', 'Hispanic...",...,To butt,Bald,30 - 32 DDD,check_box_outline_blank,check_box_outline_blank,check_box_outline_blank,"[{'by': 'diazbenjamin19', 'reviewer_reviews': ...",NaN,NaN,NaN
3,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Anjelicasworld,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Anjelicasworld.00.jpg', 'Anjelicasworld.01.j...",Have a great day,"May 26, 2019",2 hour(s) ago,4.5,"['Escort', 'Escort', 'FBSM', 'Female', '29', '...",...,To mid-back,Bald,30 - 32 D,check_box_outline_blank,check_box_outline_blank,check_box_outline_blank,"[{'by': 'Aztecs498', 'reviewer_reviews': '3', ...",ajlarson20020@gmail.com,NaN,NaN
4,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",River,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['River.00.jpg', 'River.01.jpg', 'River.02.jpg...",River awakens Phoenix retiring soon! Last chan...,"August 9, 2018",2 hour(s) ago,5.0,"['Escort', 'Escort', 'FBSM', 'Female', '29', '...",...,To shoulders,Landing strip,30 - 32 C,check_box_outline_blank,check_box_outline_blank,check_box_outline_blank,"[{'by': 'TheEvi1Twin', 'reviewer_reviews': '1'...",*Please expedite screening/peace of mind by fi...,www.experienceriver.com,NaN


## Calculate days since joined

In [43]:
import numpy as np

data_gathered = pd.to_datetime('10/12/2020 00:00') #extraction ran overnight, midnight is a reasonable simplification

df['joined_dt'] = pd.to_datetime(df['Joined'], infer_datetime_format=True) #format='%M %d, %Y', errors='coerce'
df['days_since_joined'] = data_gathered - df['joined_dt']

df.head()

,url,location,username,img_urls,img_saved,title,Joined,Bumped,stars,self_descriptors,...,Chest,Piercings,Tattoos,Smoke?,reviews,email,link,Created,joined_dt,days_since_joined
0,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Karmen2020,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Karmen2020.00.jpg', 'Karmen2020.01.jpg', 'Ka...",New Girl Karmen,"July 14, 2020",1 hour(s) ago,0.0,"['Escort', 'Escort', 'Female', '25', 'Mixed', ...",...,NaN,check_box_outline_blank,check_box_outline_blank,check_box_outline_blank,[],NaN,NaN,NaN,2020-07-14,90 days
1,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Minx_MsMariah,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Minx_MsMariah.00.jpg', 'Minx_MsMariah.01.jpg...",Secret Sunday Shenanigans,"January 27, 2019",1 hour(s) ago,5.0,"['Escort', 'Escort', 'Female', '45', 'White', ...",...,36 - 38 DD,check_box,check_box,check_box_outline_blank,"[{'by': 'Chowda', 'reviewer_reviews': '7', 'Jo...",msmariahmoon@protonmail.com,NaN,NaN,2019-01-27,624 days
2,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Abigail_Reign,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Abigail_Reign.00.jpg', 'Abigail_Reign.01.jpg...",Save Us Time And Read My Ad In Its Entirety,"May 12, 2020",1 hour(s) ago,4.5,"['Escort', 'Escort', 'Female', '30', 'Hispanic...",...,30 - 32 DDD,check_box_outline_blank,check_box_outline_blank,check_box_outline_blank,"[{'by': 'diazbenjamin19', 'reviewer_reviews': ...",NaN,NaN,NaN,2020-05-12,153 days
3,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Anjelicasworld,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Anjelicasworld.00.jpg', 'Anjelicasworld.01.j...",Have a great day,"May 26, 2019",2 hour(s) ago,4.5,"['Escort', 'Escort', 'FBSM', 'Female', '29', '...",...,30 - 32 D,check_box_outline_blank,check_box_outline_blank,check_box_outline_blank,"[{'by': 'Aztecs498', 'reviewer_reviews': '3', ...",ajlarson20020@gmail.com,NaN,NaN,2019-05-26,505 days
4,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",River,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['River.00.jpg', 'River.01.jpg', 'River.02.jpg...",River awakens Phoenix retiring soon! Last chan...,"August 9, 2018",2 hour(s) ago,5.0,"['Escort', 'Escort', 'FBSM', 'Female', '29', '...",...,30 - 32 C,check_box_outline_blank,check_box_outline_blank,check_box_outline_blank,"[{'by': 'TheEvi1Twin', 'reviewer_reviews': '1'...",*Please expedite screening/peace of mind by fi...,www.experienceriver.com,NaN,2018-08-09,795 days


## Extract self descriptors to columns

In [62]:
import ast
from collections import Counter

df_desc = df.copy()

#Self descriptors are saved in list form as raw characters,
#must evaluate literally as python list with ast.literal_eval.
df_desc['self_descriptors_literal'] = df_desc['self_descriptors'].apply(lambda x: ast.literal_eval(x))

#Save all values to a single list, then count individual values
full_list = [y for x in list(df_desc['self_descriptors_literal'].values) for y in x]
counts = Counter(full_list)
counts

Counter({'Escort': 24562,
         'Female': 12709,
         '25': 1458,
         'Mixed': 2427,
         '5 ft 6 in': 1431,
         'Incall': 10121,
         'Outcall': 10505,
         '45': 68,
         'White': 6015,
         '30': 681,
         'Hispanic': 566,
         '5 ft 4 in': 1798,
         'Toned': 1764,
         'FBSM': 4279,
         '29': 566,
         '5 ft 7 in': 1503,
         '24': 1138,
         '5 ft 5 in': 1582,
         'Curvy': 4268,
         'BDSM': 3077,
         '32': 267,
         '21': 1045,
         'Black': 1311,
         'Petite': 2379,
         '26': 1028,
         '4 ft 4 in': 3,
         '23': 1196,
         'Middle Eastern': 205,
         '28': 786,
         'Asian': 549,
         'Latina': 1520,
         '5 ft 2 in': 1452,
         '5 ft 8 in': 775,
         'TransSexual': 55,
         '5 ft 0 in': 518,
         '5 ft 3 in': 1551,
         'Native American': 122,
         '22': 1008,
         'Athletic': 1808,
         '5 ft 1 in': 690,
         '2

In [75]:
df_extract = df_desc.copy()

# Get height in inches from highly standardized values 
df_extract['height'] = df_extract['self_descriptors'].str.extract(r'(\d ft \d\d? in)')

def height_to_inches(x):
    if pd.isnull(x):
        return 0
    nums = re.findall(r"(\d+)",x) #all sequences of digits
    ft_to_inches = int(nums[0])*12
    if len(nums) > 1:
        ft_to_inches += int(nums[1])
    return ft_to_inches

df_extract['height_inches'] = df_extract['height'].apply(height_to_inches)

# Extract age as well, highly standardized
df_extract['age'] = df_extract['self_descriptors'].str.extract(r"'(\d\d)'")

# Here are extracted values as specified from the counts above, 
# From what I can tell, users may list more than one of the multiselect list,
# but they will only select one value from the other lists. 
# Therefore I will create columns for each multiselect value, 
# then separate columns for the other lists 
# with insertions conditional on matching a value.
multiselect_list = ['Incall', "Outcall", "Couple", "BDSM", "FBSM"]
physical_list = ['Athletic', 'BBW', 'Curvy', 'Petite', 'Toned']
race_list = ['Asian', 'Black', 'East Indian', 'Hispanic', 'Latina', 'Middle Eastern', 'Mixed', 'Native American', 'White']
gender_list = ["'Female'", "'Male'", 'TransSexual']
               
# Create new columns for each item in the multiselect list if it exists
for item in multiselect_list:
    df_extract[item] = df_extract['self_descriptors'].apply(lambda x: 1 if item in x else 0)

# Meta lists
single_assigns = [physical_list, race_list, gender_list]
single_assign_names = ["physical", "race", "gender"]

# For each list given above create a new column,
# and fill that column only when a given option is found
for i in range(len(single_assign_names)):
    
    #get current list and column name
    current_list = single_assigns[i]
    current_column = single_assign_names[i]
    
    #create the column with empty strings
    df_extract[current_column] = ""

    #check if each item in this list is found
    for item in current_list:
        
        def assign_item_else_nothing(x):
            """Function to only update to value if exists, otherwise
            give existing value."""
            #as a row, column names are not attached, 
            #must find column numbers
            dfcols = list(df_extract.columns)
            col_self_desc = int(dfcols.index('self_descriptors'))
            col_phys = int(dfcols.index(current_column))
            self_desc_val = x[col_self_desc]

            internal_item = item

            if str(internal_item).lower() in self_desc_val.lower():
                return str(internal_item).replace("'","")
            else:
                return x[col_phys].replace("'","")
            
        # apply given function on every row of the database
        df_extract[current_column] = df_extract.apply(assign_item_else_nothing, axis = 1)

df_extract.head()

,url,location,username,img_urls,img_saved,title,Joined,Bumped,stars,self_descriptors,...,height_inches,age,Incall,Outcall,Couple,BDSM,FBSM,physical,race,gender
0,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Karmen2020,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Karmen2020.00.jpg', 'Karmen2020.01.jpg', 'Ka...",New Girl Karmen,"July 14, 2020",1 hour(s) ago,0.0,"['Escort', 'Escort', 'Female', '25', 'Mixed', ...",...,66,25,1,1,0,0,0,,Mixed,Female
1,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Minx_MsMariah,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Minx_MsMariah.00.jpg', 'Minx_MsMariah.01.jpg...",Secret Sunday Shenanigans,"January 27, 2019",1 hour(s) ago,5.0,"['Escort', 'Escort', 'Female', '45', 'White', ...",...,66,45,1,0,0,0,0,,White,Female
2,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Abigail_Reign,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Abigail_Reign.00.jpg', 'Abigail_Reign.01.jpg...",Save Us Time And Read My Ad In Its Entirety,"May 12, 2020",1 hour(s) ago,4.5,"['Escort', 'Escort', 'Female', '30', 'Hispanic...",...,64,30,1,1,0,0,0,Toned,Hispanic,Female
3,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Anjelicasworld,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Anjelicasworld.00.jpg', 'Anjelicasworld.01.j...",Have a great day,"May 26, 2019",2 hour(s) ago,4.5,"['Escort', 'Escort', 'FBSM', 'Female', '29', '...",...,67,29,1,1,0,0,1,Toned,White,Female
4,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",River,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['River.00.jpg', 'River.01.jpg', 'River.02.jpg...",River awakens Phoenix retiring soon! Last chan...,"August 9, 2018",2 hour(s) ago,5.0,"['Escort', 'Escort', 'FBSM', 'Female', '29', '...",...,66,29,1,1,0,0,1,Toned,Mixed,Female


## Extract No List to columns

In [346]:
#Process No List

df_no = df_extract.copy()
df_no = df_no.drop(['reviews'], axis=1)

#extract nolist to columns
df_no['nolist'] = df_no['nolist'].apply(lambda x: ast.literal_eval(x))

#Save all values to a single list, then count individual values
all_nos = list(set([y for x in list(df_no['nolist'].values) for y in x]))

# Create new columns for each item in the multiselect list if it exists
for item in all_nos:
    df_no["no_" + item] = df_no['nolist'].apply(lambda x: 1 if item in x else 0)
    
df_no.head()

,url,location,username,img_urls,img_saved,title,Joined,Bumped,stars,self_descriptors,...,no_FOV,no_D,no_FIV,no_CIM,no_T,no_LFK,no_DFK,no_COF,no_BBBJ,no_CBJ
0,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Karmen2020,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Karmen2020.00.jpg', 'Karmen2020.01.jpg', 'Ka...",New Girl Karmen,"July 14, 2020",1 hour(s) ago,0.0,"['Escort', 'Escort', 'Female', '25', 'Mixed', ...",...,1,0,1,1,0,1,1,1,1,0
1,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Minx_MsMariah,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Minx_MsMariah.00.jpg', 'Minx_MsMariah.01.jpg...",Secret Sunday Shenanigans,"January 27, 2019",1 hour(s) ago,5.0,"['Escort', 'Escort', 'Female', '45', 'White', ...",...,0,0,0,0,0,0,0,0,0,0
2,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Abigail_Reign,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Abigail_Reign.00.jpg', 'Abigail_Reign.01.jpg...",Save Us Time And Read My Ad In Its Entirety,"May 12, 2020",1 hour(s) ago,4.5,"['Escort', 'Escort', 'Female', '30', 'Hispanic...",...,0,0,0,1,0,0,0,1,0,0
3,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Anjelicasworld,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Anjelicasworld.00.jpg', 'Anjelicasworld.01.j...",Have a great day,"May 26, 2019",2 hour(s) ago,4.5,"['Escort', 'Escort', 'FBSM', 'Female', '29', '...",...,0,0,1,1,0,0,0,1,0,0
4,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",River,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['River.00.jpg', 'River.01.jpg', 'River.02.jpg...",River awakens Phoenix retiring soon! Last chan...,"August 9, 2018",2 hour(s) ago,5.0,"['Escort', 'Escort', 'FBSM', 'Female', '29', '...",...,0,0,0,0,0,0,0,0,0,0


## Extract Payment Types Accepted to columns

In [375]:
#Process Payment Types Accepted

df_payment = df_no.copy()

#extract nolist to columns
def split_if_any(x):
    x = str(x).replace(" ,",", ").strip().lower()
    x = x.replace("credit/debit", "cr_db")
    if x and "," in x:
        x = [a.replace(" ", "_") for a in x.split(", ")]
        return x
    return ["na"]

df_payment['payment_types_accepted'] = df_payment['payment_types_accepted'].apply(split_if_any)

#Save all values to a single list, then count individual values
all_payments = list(set([y for x in list(df_payment['payment_types_accepted'].values) for y in x if x]))
#all_payments

# Create new columns for each item in the multiselect list if it exists
for item in all_payments:
    df_payment["pta_" + item] = df_payment['payment_types_accepted'].apply(lambda x: 1 if item in x else 0)
    
df_payment.head()

,url,location,username,img_urls,img_saved,title,Joined,Bumped,stars,self_descriptors,...,pta_cashapp,pta_zelle,pta_cr_db_card,pta_google_wallet,pta_paypal,pta_cash,pta_bitcoin,pta_litecoin,pta_apple_pay,pta_ethereum
0,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Karmen2020,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Karmen2020.00.jpg', 'Karmen2020.01.jpg', 'Ka...",New Girl Karmen,"July 14, 2020",1 hour(s) ago,0.0,"['Escort', 'Escort', 'Female', '25', 'Mixed', ...",...,1,1,0,0,0,1,0,0,0,0
1,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Minx_MsMariah,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Minx_MsMariah.00.jpg', 'Minx_MsMariah.01.jpg...",Secret Sunday Shenanigans,"January 27, 2019",1 hour(s) ago,5.0,"['Escort', 'Escort', 'Female', '45', 'White', ...",...,0,0,0,0,1,1,0,0,0,0
2,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Abigail_Reign,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Abigail_Reign.00.jpg', 'Abigail_Reign.01.jpg...",Save Us Time And Read My Ad In Its Entirety,"May 12, 2020",1 hour(s) ago,4.5,"['Escort', 'Escort', 'Female', '30', 'Hispanic...",...,1,1,0,0,0,1,0,0,0,0
3,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",Anjelicasworld,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['Anjelicasworld.00.jpg', 'Anjelicasworld.01.j...",Have a great day,"May 26, 2019",2 hour(s) ago,4.5,"['Escort', 'Escort', 'FBSM', 'Female', '29', '...",...,1,1,0,0,1,1,0,0,1,0
4,https://privatedelights.ch/USA/Arizona/Phoenix...,"Phoenix, Arizona",River,['https://dmacnjnna4ptc.cloudfront.net/large/a...,"['River.00.jpg', 'River.01.jpg', 'River.02.jpg...",River awakens Phoenix retiring soon! Last chan...,"August 9, 2018",2 hour(s) ago,5.0,"['Escort', 'Escort', 'FBSM', 'Female', '29', '...",...,1,0,0,0,0,1,0,0,0,0


In [376]:
df_payment.to_csv("User_data_features_extracted.csv")

## Donation Analysis and Extraction

In [328]:
df_donation = df_extract.copy()
#list(df_desc['donation_options'].values)

#Self descriptors are saved in list form as raw characters,
#must evaluate literally as python list with ast.literal_eval.
def liteval_if_list(x):
    x = str(x)
    if ']' in x and '[' in x:
        return ast.literal_eval(x)
    else:
        return [[x]]
df_donation['donation_lists'] = df_donation['donation_options'].apply(liteval_if_list)

# "Explode" or make a new row for each value in donation list, repeat other columns
df_donation_only = df_donation[['url','username','donation_lists']].explode('donation_lists')
df_donation_only = df_donation_only.dropna() #remove rows with nothing
df_donation_only = df_donation_only.reset_index() #reset index to "set" data form

#if there is a multiplication provided with an x, calculate the multiplied value
def multiplyers(item, xsearch):
    xsearch = [x.replace("@", ".") for x in xsearch[0]]
    fullmatch = 'x'.join(xsearch)
    outnum = int(xsearch[0]) * int(xsearch[1])
    return [fullmatch, outnum]

#clean up values in donation raw text
def clean_donations(x):
    outlist = []
    
    for item in x:
        item = re.sub("(\d{3}[\-\/\)]?\d{3}[\-\/\)]?\d{4})", "", item) #remove phone numbers
        item = re.sub("(per|rate|is) ", " ", item.lower().strip()) #remove per and rate
        item = re.sub(r"\$(\d+\.?\d?)k", r"\g<1>000", item) #convert $1k to 1000
        item = re.sub(r"(\d)\,(\d\d\d)", r"\g<1>\g<2>", item) #convert 1,000 to 1000
        item = re.sub(r"(\d+)\-(\d+)", r" \g<1> ", item) #for each number range, reduce to first value only
        
        #convert decimal periods to @ to sidestep accidental adjustment during processing
        item = item.replace(".00", "").replace(r"1/2", "0.5").replace(".","@")
        
        item = re.sub(r"([:* \/\~\(]|\@\@)", " ", item) #convert unusual/multiple spacers to a space
        item = re.sub(r"[^a-z0-9 \,\/\-\@]", "", item) # remove unexpected characters
        
        #convert 100x2 to 200 etc. 
        xsearch = re.findall(r"([\d\@]+)x([\d\@]+)", item)
        if xsearch:
            xsearch = multiplyers(item, list(xsearch))
            item = item.replace(xsearch[0], str(xsearch[1]))
        
        #item = re.sub(r"(full)", " 1 ", item) #convert full to 1, i.e. full hour = 1 hour
        
        item = re.sub(r"(qk|quick)", " 0@25 hr ", item) #convert jargon qk to 15 min
        item = re.sub(r"(hhr|hh|half hour|half hr)", " 0@5 hr ", item) #convert "half hour" jargon to 0.5 hours
        item = re.sub(r"(half)", " 0@5 ", item) #convert remaining "half" to 0.5
        item = re.sub(r"(h[ours\.\@]+? |hours?)", " hr ", item) #make hour text consistent
        item = re.sub(r"(\d{3,})[\s|\/]+hr", " \g<1> 1 hr ", item) #dollars per hour to dollars per 1 hour
        item = re.sub(r"(min)", " \g<1> ", item) #add space around minutes quantifier
        item = re.sub(r"([a-z]+)([\d\@]+)([a-z]+)", r" \g<1> \g<2> \g<3> ", item) #add space around numbers in text
        item = re.sub(r"\/?([\d\@]+)[ \/\@\-]*(h[a-z]*|inout|qk|min[a-z]*)\/?", r" \g<2> \g<1> ", item) #rearrange number-text to text number
        item = re.sub(r"\s{2,}", " ", item).strip().lstrip(",") # simplify spaces and strip
        item = re.split(r"([a-zA-Z]+|[\d\@]+)[ \-\/\,]+([\d\@]+|hr|qk|min[a-z]*)", item)
        
        #ensure lists are joined and items appended
        if type(item) == list:
            outlist += item
        else:
            outlist.append(item)
    outlist = [re.sub(r"[\/\-]", "", x.strip().rstrip("s")).replace("@",".") for x in outlist]
    outlist = [x for x in outlist if x]
    return outlist
    
# (?:mins?|hours?|gfe|days?|week) $?\d+
df_donation_only['donation_list2'] = df_donation_only['donation_lists'].apply(clean_donations)

# Create new columns for each item in the multiselect list if it exists
# based on raw extracted string
bool_cols = ["incall", "outcall", "fly", "gfe", "greek"]
for item in bool_cols:
    df_donation_only[item] = df_donation_only['donation_lists'].apply(lambda x: 1 if item in ''.join(x).lower() else 0)

# Create new columns for each item in the multiselect list if it exists
# based on cleaned up strings
bool_cols = ["min", "hr", "qk", "day", "overnight", "week"]
for item in bool_cols:
    df_donation_only[item] = df_donation_only['donation_list2'].apply(lambda x: 1 if item in x else 0)

#extract all numbers from text 
def num_only_list(x):
    numlist = []
    for item in x:
        numbers = re.findall(r'(\d+\.\d+|\d+)', item)
        if numbers:
            numlist += numbers
    
    numlist = [float(n) for n in numlist if float(n) > 0]
    
    # if empty list, or the max value is too low to have $, exit
    if not numlist or max(numlist) < 15:
        return []
    if len(numlist) % 2 != 0:
        numlist.append(1.0)
    
    numlist = sorted(numlist)
    return numlist
df_donation_only['num_only_list'] = df_donation_only['donation_list2'].apply(num_only_list)

prior_colnames = list(df_donation_only.columns)

# evenly split up number list in 2 for quantifiers and prices
def split_list(x):
    if type(x) != list or not x or len(x) %2 > 0:
        return [], []
    length = len(x)
    mid = int(length / 2)
    return x[:mid], x[mid:]

splitup = df_donation_only['num_only_list'].apply(split_list)

#join it all together
new_colnames = ["quantities", "prices"]
split_df = pd.DataFrame(splitup).reset_index()
split_df["quantities"] = split_df['num_only_list'].apply(lambda x: x[0])
split_df["prices"] = split_df['num_only_list'].apply(lambda x: x[1])

df_donations_joined = pd.concat([df_donation_only, split_df[["quantities", "prices"]]], axis=1, ignore_index=True)
df_donations_joined.columns = prior_colnames + new_colnames
df_donations_joined.head(55)

startval = 10000
rangevals = list(range(startval,startval+59))
df_donation_only.iloc[rangevals]


,index,url,username,donation_lists,donation_list2,incall,outcall,fly,gfe,greek,min,hr,qk,day,overnight,week,num_only_list
10000,3563,https://privatedelights.ch/USA/California/Sacr...,LovelyZade,"[, 👉 Booking INCALL this wk (530) 404-0036]","[booking incall th, wk, 530, 404]",1,0,0,0,0,0,0,0,0,0,0,"[404.0, 530.0]"
10001,3563,https://privatedelights.ch/USA/California/Sacr...,LovelyZade,"[, 20 min $100 BloNgo]","[min, 20, 100 blongo]",0,0,0,0,0,1,0,0,0,0,0,"[20.0, 100.0]"
10002,3563,https://privatedelights.ch/USA/California/Sacr...,LovelyZade,"[GFE+ Incall / CarFun Special, 30min-$140 (Reg...","[gfe incall carfun special, min, 30, 140, reg,...",1,0,0,1,0,1,0,0,0,0,0,"[1.0, 30.0, 140.0, 180.0]"
10003,3563,https://privatedelights.ch/USA/California/Sacr...,LovelyZade,"[GFE+ Incall / CarFun Special, 60min-$220 (Reg...","[gfe incall carfun special, min, 60, 220, reg,...",1,0,0,1,0,1,0,0,0,0,0,"[1.0, 60.0, 220.0, 260.0]"
10004,3563,https://privatedelights.ch/USA/California/Sacr...,LovelyZade,"[Outcall donation + Uber/Lyft/Pkup, 30min-180]","[outcall donation uber lyft pkup, min, 30, 180]",0,1,0,0,0,1,0,0,0,0,0,"[30.0, 180.0]"
10005,3563,https://privatedelights.ch/USA/California/Sacr...,LovelyZade,"[Outcall donation + Uber/Lyft/Pkup, 60min-250]","[outcall donation uber lyft pkup, min, 60, 250]",0,1,0,0,0,1,0,0,0,0,0,"[60.0, 250.0]"
10006,3564,https://privatedelights.ch/USA/California/Sacr...,Lilylove,"[⚠️Screening is a must ⚠️, 💕15 mins, 140]","[screening a must, min, 15, 140]",0,0,0,0,0,1,0,0,0,0,0,"[15.0, 140.0]"
10007,3564,https://privatedelights.ch/USA/California/Sacr...,Lilylove,"[⚠️Screening is a must ⚠️, 💕30 mins, 200]","[screening a must, min, 30, 200]",0,0,0,0,0,1,0,0,0,0,0,"[30.0, 200.0]"
10008,3564,https://privatedelights.ch/USA/California/Sacr...,Lilylove,"[⚠️Screening is a must ⚠️, 💕60 mins, 300]","[screening a must, min, 60, 300]",0,0,0,0,0,1,0,0,0,0,0,"[60.0, 300.0]"
10009,3564,https://privatedelights.ch/USA/California/Sacr...,Lilylove,"[⚠️Screening is a must ⚠️, 💕90 mins, 360]","[screening a must, min, 90, 360]",0,0,0,0,0,1,0,0,0,0,0,"[90.0, 360.0]"


In [327]:
df_donations_joined.to_csv("Donation_Extraction.csv")

## Review Extraction

In [340]:
df_reviews = df[["url", "reviews"]].copy() #make a new copy to avoid copy slicing issues
df_reviews['reviews'] = df_reviews['reviews'].apply(liteval_if_list) #literal evaluation of saved list
df_reviews = df_reviews.dropna().explode('reviews').dropna() #drop NAs, explode reviews into rows instead of list items

#convert dictionary from list to named columns for each row
df_reviews_to_cols = pd.concat([df_reviews.drop(['reviews'], axis=1), df_reviews['reviews'].apply(pd.Series)], axis=1)
df_reviews_to_cols.head()

,url,by,reviewer_reviews,Joined,stars,Reviewed,Visit Date,provider_replied,review
1,https://privatedelights.ch/USA/Arizona/Phoenix...,Chowda,7,"May 12, 2020",5.0,"October 4, 2020",August 2020,check_circle,Double the fun
1,https://privatedelights.ch/USA/Arizona/Phoenix...,Deemee100,1,"April 15, 2020",10.0,"October 4, 2020",October 2020,check_circle,Amazing
1,https://privatedelights.ch/USA/Arizona/Phoenix...,Tetherceo,9,"November 6, 2018",15.0,"September 10, 2020",August 2020,check_circle,Double trouble
1,https://privatedelights.ch/USA/Arizona/Phoenix...,MrRed,7,"February 27, 2020",20.0,"August 31, 2020",August 2020,check_circle,So so good.
1,https://privatedelights.ch/USA/Arizona/Phoenix...,Sfnx,9,"January 30, 2020",25.0,"July 6, 2020",June 2020,check_circle,Never experienced anything like her!!!


In [341]:
df_reviews_to_cols.to_csv("reviews.csv")